## 1. 定义问题

BigMart收集了2013年10家商店1559种产品的销售数据，每家商店和每种产品的特征已经明确定义。

目标：建立回归模型，预测产品销量。

参考：
* [AnalyticsVidhya](https://www.analyticsvidhya.com/blog/2016/02/bigmart-sales-solution-top-20/)
* [预测BigMartSales](https://medium.com/diogo-menezes-borges/project-1-bigmart-sale-prediction-fdc04f07dc1e)

首先我们应该思考，哪些因素可能影响产品销量，了解问题是解决问题最重要的一步。

* 产品价格：如果商品是必需品，价格越高销量越低，价格越低销量越高。
* 产品属性：必需品，还是奢侈品。
* 品牌知名度：越是知名的产品，顾客信任度越高，销量也会越高。
* 替代品价格：替代品价格越高，销量越高，相反销量越低。
* 促销/打折：营销活动一般能够促进销量。
* 商品摆放在店面的位置：相对于不显眼的位置，放在显眼位置的商品销量可能更高。
* 店面的大小：商店规模越大，能够吸引的人流更多，商品销量也可能更高。
* 附近类似商家的数量：竞争越激烈，商品越难销售。
* 商店地段：人口密度越高，需求也会越大，进而推动销量。
* 所在城市的规模：相对于三线城市，一线城市居民的收入会更高，意味着购买更多的商品。

## 2. 探索数据

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train = pd.read_csv("data/sales_train.csv")
test = pd.read_csv("data/sales_test.csv")

# 先合并训练集和检验集，方便进行特征工程
train["source"] = "train"
test["source"] = "test"
data = pd.concat([train, test], ignore_index=True, sort=False)

data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [3]:
print("training set:", train.shape)
print("test set:", test.shape)

training set: (8523, 13)
test set: (5681, 12)


查看是否存在缺失值。

In [4]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

'Item_Weight'和'Outlet_Size'两个变量存在很多缺失值。产品重量可以忽略，但商店大小有可能很重要，如何处理缺失值？

计算数值变量的描述统计量。

In [5]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


'Item_Visibility'的最小值为零，但产品展示面积不可能为零，有可能是记录错误，如何处理？

查看分类变量的类别。

In [6]:
# 根据数据类型推断是否为分类变量
cat_features = [idx for idx,val in zip(data.dtypes.index, data.dtypes) if val == "object"]
cat_features

# 忽略产品ID和商店ID，查看其余分类变量的类别
cat_features.remove("Item_Identifier")
cat_features.remove("Outlet_Identifier")
cat_features.remove("source")

for feature in cat_features:
    print(f"Feature name: {feature}")
    print(data[feature].value_counts())
    print("")

Feature name: Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Feature name: Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Feature name: Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Feature name: Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Feature name: Outlet_Type
Supermarket Type1    9294
Grocery Store        1805
Supermarket 

脂肪含量的类别有重复标签，'Low Fat','low fat','LF'都指低脂商品，'Regular'和'reg'指正常脂肪含量。

## 3. 数据清洗

数据集包含1559种商品，用特定种类商品的平均重量填充缺失值。

In [7]:
items_mean_weight = data.groupby("Item_Identifier")["Item_Weight"].mean()
items_weight = pd.Series(data["Item_Weight"], index=data["Item_Identifier"])
items_weight.fillna(items_mean_weight, inplace=True)
data["Item_Weight"] = items_weight.values
print("NAs for Item_Weight: %d" % items_weight.isnull().sum())

NAs for Item_Weight: 0


用特定种类商店出现次数最多的规模(众数)替代商店规模的缺失值。

众数(mode): 分类变量中出现次数最多的类别。

In [8]:
data.groupby("Outlet_Type")["Outlet_Size"].value_counts(dropna=False)

Outlet_Type        Outlet_Size
Grocery Store      NaN             925
                   Small           880
Supermarket Type1  Small          3100
                   NaN            3091
                   High           1553
                   Medium         1550
Supermarket Type2  Medium         1546
Supermarket Type3  Medium         1559
Name: Outlet_Size, dtype: int64

In [9]:
def cal_mode(ser, dropna=True):
    """计算众数：分类变量中出现次数最多的类别"""
    counts = ser.value_counts(dropna=dropna)
    counts.sort_values(ascending=False)
    return counts.index[0]

outlets_size_mode = data.groupby("Outlet_Type")["Outlet_Size"].apply(cal_mode)
outlets_size = pd.Series(data["Outlet_Size"], index=data["Outlet_Type"])
outlets_size.fillna(outlets_size_mode, inplace=True)
data["Outlet_Size"] = outlets_size.values

print("NAs for Outlet Size: %d" % data["Outlet_Size"].isnull().sum())

NAs for Outlet Size: 0


产品可见度不可能为零，把零值视为缺失值，用特定商品的平均可见度填充缺失值。

In [10]:
items_visibility = pd.Series(data["Item_Visibility"], index=data["Item_Identifier"])
items_visibility.replace(0, np.nan, inplace=True)
items_mean_visibility = data.groupby("Item_Identifier")["Item_Visibility"].mean()
items_visibility.fillna(items_mean_visibility, inplace=True)

data["Item_Visibility"] = items_visibility.values
data["Item_Visibility"].describe()

count    14204.000000
mean         0.065953
std          0.044145
min          0.003895
25%          0.030728
50%          0.055485
75%          0.091792
max          0.211315
Name: Item_Visibility, dtype: float64

统一脂肪含量的所有类别。

In [11]:
mappings = {
    "low fat": "Low Fat",
    "LF": "Low Fat",
    "reg": "Regular"
}
data["Item_Fat_Content"] = data["Item_Fat_Content"].replace(mappings)
data["Item_Fat_Content"].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

## 4. 特征工程

商店开业年份对预测商品销量的意义不大，考虑转化为'经营年数'，经营时间越长，客户信赖度越高，预期和产品销量正相关。

In [12]:
data["Outlet_years"] = 2013 - data["Outlet_Establishment_Year"]
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Outlet_years
count,14204.000000,14204.000000,14204.000000,14204.000000,8523.000000,14204.000000
mean,12.793380,0.065953,141.004977,1997.830681,2181.288914,15.169319
std,4.651716,0.044145,62.086938,8.371664,1706.499616,8.371664
min,4.555000,0.003895,31.290000,1985.000000,33.290000,4.000000
25%,8.710000,0.030728,94.012000,1987.000000,834.247400,9.000000
50%,12.600000,0.055485,142.247000,1999.000000,1794.331000,14.000000
75%,16.750000,0.091792,185.855600,2004.000000,3101.296400,26.000000
max,21.350000,0.211315,266.888400,2009.000000,13086.964800,28.000000


将分类变量转化为数值变量，使用独热编码(one-hot encoding)技术。为了避免虚拟变量陷阱，对k个类别的分类变量，引入(k-1)个虚拟变量。

所谓独热编码，即把具有m个类别的分类特征转化为$n*m$的二元矩阵，n是观测值的数量，m是类别的数量，对于特定的观测值，如果某类别出现记为1，否则记为0.

Pandas.get_dummies()可以轻易实现独热编码。

In [13]:
# 最终要提交的submission.csv要求包含'Outlet_Identifier'
# 我们想将分类特征'Outlet_Identifier'进入模型，所以先创建副本'Outlet_ID'
# 将'Outlet_ID'转化为数字变量，这就同时保留了'Outlet_Identifier'
data["Outlet_ID"] = data["Outlet_Identifier"]

columns_to_encode = [
    "Item_Fat_Content",
    "Item_Type",
    "Outlet_ID",
    "Outlet_Size",
    "Outlet_Location_Type",
    "Outlet_Type"
]

data = pd.get_dummies(data, columns=columns_to_encode, drop_first=True)

查看所有特征和数据类型。

In [14]:
data.dtypes

Item_Identifier                     object
Item_Weight                        float64
Item_Visibility                    float64
Item_MRP                           float64
Outlet_Identifier                   object
Outlet_Establishment_Year            int64
Item_Outlet_Sales                  float64
source                              object
Outlet_years                         int64
Item_Fat_Content_Regular             uint8
Item_Type_Breads                     uint8
Item_Type_Breakfast                  uint8
Item_Type_Canned                     uint8
Item_Type_Dairy                      uint8
Item_Type_Frozen Foods               uint8
Item_Type_Fruits and Vegetables      uint8
Item_Type_Hard Drinks                uint8
Item_Type_Health and Hygiene         uint8
Item_Type_Household                  uint8
Item_Type_Meat                       uint8
Item_Type_Others                     uint8
Item_Type_Seafood                    uint8
Item_Type_Snack Foods                uint8
Item_Type_S

剔除多余的特征，重新生成训练集和检验集。

In [15]:
data.drop(columns="Outlet_Establishment_Year", inplace=True)

train = data.query("source == 'train'").copy()
test = data.query("source == 'test'").copy()

train.drop(columns="source", inplace=True)
test.drop(columns=["Item_Outlet_Sales", "source"], inplace=True)

train.to_csv("data/sales_train_modify.csv", index=False)
test.to_csv("data/sales_test_modify.csv", index=False)

## 5. 建立模型

### 5.1 多元线性回归

建立一个多元线性回归模型，作为基准模型(baseline model).

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
def build_model(estimator, train_df, test_df, feature_names,
                target_name, output):
    """建立模型
    
    1. 创建模型，拟合数据。
    2. 10折交叉验证，评估预测能力。
    3. 可视化回归系数。
    4. 样本外预测，生成submission.csv.
    """
    # 划分特征和目标变量
    X_train = train_df[feature_names]
    y_train = train_df[target_name]
    
    # 拟合数据
    estimator.fit(X_train, y_train)
    
    # 交叉验证
    # 调用sklearn.metrics.SCORERS.keys()查看所有可用的评分指标
    # neg_mean_squared_error = -MSE, sklearn规定所有评分指标
    # 都服从一个规则：评分越高模型越好，所以返回负的均方误差
    scores = {
        "RMSE": "neg_mean_squared_error",
        "R2": "r2"
    }
    print("10-fold Cross Validation")
    for score_name,score_func in scores.items():
        cv_scores = cross_val_score(estimator, X_train, y_train,
                                    cv=10, scoring=score_func)
        if score_name == "RMSE":
            cv_scores = np.sqrt(np.abs(cv_scores))
        print(f"{score_name}: mean={np.mean(cv_scores):.3f} " \
              f"min={np.min(cv_scores):.3f} "\
              f"max={np.max(cv_scores):.3f}")

    # 生成submission.csv
    X_test = test_df[feature_names]
    y_pred = estimator.predict(X_test)
    columns = ["Item_Identifier", "Outlet_Identifier", target_name]
    test_df[target_name] = y_pred
    test_df.to_csv(output, columns=columns, index=False)
    print(f"Export submission file: {output}")

In [18]:
feature_names = train.columns.drop([
    "Item_Identifier", "Outlet_Identifier",
    "Item_Outlet_Sales"
])
target_name = "Item_Outlet_Sales"

model_0 = LinearRegression()

build_model(model_0, train, test, feature_names, target_name, "bigMart_model0.csv")

10-fold Cross Validation
RMSE: mean=1132.264 min=1110.953 max=1164.984
R2: mean=0.558 min=0.509 max=0.582
Export submission file: bigMart_model0.csv


### 5.2 岭回归

In [19]:
from sklearn.linear_model import Ridge

model_1 = Ridge(alpha=100)

build_model(model_1, train, test, feature_names, target_name, "bigMart_model1.csv")

10-fold Cross Validation
RMSE: mean=1135.194 min=1114.256 max=1167.016
R2: mean=0.556 min=0.509 max=0.578
Export submission file: bigMart_model1.csv


## 5.3 Lasso回归

In [20]:
from sklearn.linear_model import Lasso

In [21]:
model_2 = Lasso(alpha=10)
build_model(model_2, train, test, feature_names, target_name, "bigMart_model2.csv")

10-fold Cross Validation
RMSE: mean=1132.022 min=1112.325 max=1164.586
R2: mean=0.559 min=0.512 max=0.580
Export submission file: bigMart_model2.csv
